## Used care listings
We are working with dataset fof used cars from eBay 'Kleinanzeigen', a classfieds section of the German eBay website. The dataset was originally scraped and uploaded to Kaggle, now with a few modifications from the original dataset:

+ now sampled 50,000 data points from the full dataset to ensure the code runs quickly in hosted enviroment
+ Dirtied the dataset a bit to more closely resemable what would have been expected from a scraped dataset

The aim of this proejct is to clearn the data and analyze the included used car listings.


In [1]:
import pandas as pd

dateparser = lambda x:pd.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
autos = pd.read_csv('autos.csv',encoding='Latin-1',parse_dates=['dateCrawled','dateCreated','lastSeen'],date_parser=dateparser)
autos.head()



,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01,0,39218,2016-04-01 14:38:50


In [2]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null datetime64[ns]
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null datetime64[ns]
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50

## Observations
+ data volume of 50000
+ memory usage of 7.6+ MB
+ 20 columns of mostly strings
+ some columns have null values, not more than 20% of the data
+ The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

In [3]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [4]:
autos = autos.rename(columns={'dateCrawled':'date_crawled', 
                              'yearOfRegistration':'registration_year',
                      'monthOfRegistration':'registration_month',
                      'notRepairedDamage':'unrepaired_damage',
                      'dateCreated':'ad_created',
                      'lastSeen':'last_seen',
                      'postalCode':'postal_code',
                     'nrOfPictures':'number_pictures',
                     'fuelType':'fuel_type',
                     'powerPS':'power_ps',
                     'vehicleType':'vehicle_type',
                     'offerType':'offer_type'})

In [5]:
autos.head()


,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,number_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01,0,39218,2016-04-01 14:38:50


Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to e more descriptive

In [6]:
autos.unrepaired_damage



0        nein
1        nein
2        nein
3        nein
4        nein
5         NaN
6         NaN
7        nein
8        nein
9        nein
10       nein
11        NaN
12       nein
13       nein
14        NaN
15       nein
16       nein
17       nein
18        NaN
19       nein
20        NaN
21       nein
22       nein
23       nein
24       nein
25         ja
26       nein
27        NaN
28         ja
29       nein
         ... 
49970    nein
49971    nein
49972      ja
49973    nein
49974    nein
49975    nein
49976    nein
49977    nein
49978      ja
49979    nein
49980     NaN
49981    nein
49982     NaN
49983     NaN
49984     NaN
49985     NaN
49986    nein
49987    nein
49988    nein
49989      ja
49990    nein
49991     NaN
49992    nein
49993     NaN
49994    nein
49995    nein
49996    nein
49997    nein
49998    nein
49999    nein
Name: unrepaired_damage, Length: 50000, dtype: object

may be there is a need to thoroughly look through the entry representations of the difference between 'nein' and NaN entries

In [7]:
autos.describe()

,registration_year,power_ps,registration_month,number_pictures,postal_code
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


## Observations
+ let's investigate more into power_ps
+ seems registration year and month might be stored as strings and may need some cleaning
+ We may also have to clean up the trailing longs into floats in each column depending on our usage


+ There are a number of text columns where all (or nearly all) of the values are the same:
+ + seller
+ +  offer_type
+ The num_photos column looks odd, we'll need to investigate this further.

In [8]:
autos = autos.drop(['seller','offer_type','number_pictures'],axis=1)

In [9]:
# import re

# autos.loc[:,['price','odometer']].replace(to_replace='\$([0-9,\.]+).*', value=r'\1',regex=True,inplace=True)


In [10]:
autos['price'] = [i.replace('$','').replace(',','') if '$' in i else i for i in autos['price']]
autos['price'] = autos['price'].astype('int')

In [11]:
autos['odometer'] = [i.replace('km','').replace(',','') if 'km' in i else i for i in autos['odometer']]
autos['odometer'] = autos['odometer'].astype('int')

In [12]:
autos[['price','odometer']].dtypes

price       int64
odometer    int64
dtype: object

In [13]:
autos = autos.rename(columns={'odometer':'odometer_km'})

In [14]:
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01,39218,2016-04-01 14:38:50


went ahead and took out some unnecessary characters to more easily manipulate data

In [15]:
print(autos["price"].unique().shape)
print(autos["price"].describe())
autos["price"].value_counts().head(20)

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
Name: price, dtype: int64

In [16]:
autos.odometer_km.value_counts()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64

In [17]:
autos.price.value_counts()

0         1421
500        781
1500       734
2500       643
1000       639
1200       639
600        531
800        498
3500       498
2000       460
999        434
750        433
900        420
650        419
850        410
700        395
4500       394
300        384
2200       382
950        379
1100       376
1300       371
3000       365
550        356
1800       355
5500       340
1250       335
350        335
1600       327
1999       322
          ... 
46200        1
29600        1
13480        1
21700        1
7373         1
3279         1
4286         1
188          1
17830        1
9130         1
910          1
238          1
2671         1
69900        1
151990       1
2479         1
4510         1
86500        1
47499        1
16998        1
27299        1
41850        1
4780         1
686          1
6495         1
20790        1
8970         1
846          1
2895         1
33980        1
Name: price, Length: 2357, dtype: int64

In [18]:
autos[['price','odometer_km']].sort_values(by=['price','odometer_km'],ascending=False)

,price,odometer_km
39705,99999999,150000
42221,27322222,40000
27371,12345678,150000
39377,12345678,150000
47598,12345678,150000
24384,11111111,150000
2897,11111111,50000
11137,10000000,100000
47634,3890000,5000
7814,1300000,50000


In [19]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

In [20]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

There are a number of listings with prices below \$30, including about 1,500 at \$0. There are also a small number of listings with very high values, including 14 at around or over $1 million.

Given that eBay is an auction site, there could legitimately be items where the opening bid is \$1. We will keep the \$1 items, but remove anything above \$350,000, since it seems that prices increase steadily to that number and then jump up to less realistic numbers.

In [21]:
# autos = autos[(autos[['price','odometer_km']] > 0).all(axis=1) & (autos[['price','odometer_km']] < 169000).all(axis=1)]

In [22]:
autos = autos[autos['price'].between(1,351000)]
autos.price.describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

# cleaning summary
+ did a sort values of both columns
+ found repeating dubious integer values, went ahead to index in to smallest range out of those values 


## Exploring the date columns
There are a number of columns with date information:

+ date_crawled
+ registration_month
+ registration_year
+ ad_created
+ last_seen
These are a combination of dates that were crawled, and dates with meta-information from the crawler. The non-registration dates are stored as strings.

We'll explore each of these columns to learn more about the listings.

In [23]:
autos = autos.sort_values(by=['date_crawled','ad_created','last_seen'],ascending=True)


sorted autos by date_crawled, ad_created and last_seen from ascending order 

In [24]:
autos['date_crawled'] = pd.to_datetime(autos['date_crawled']).dt.date

In [25]:
autos['last_seen'] = pd.to_datetime(autos['last_seen']).dt.date

changed last_seen and date_crawled columns to datetime date objects instead of time stamp for percentage distrubutions 

In [26]:
autos['date_crawled'].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [27]:
autos['ad_created'].value_counts(normalize=True,dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

In [28]:
autos['last_seen'].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [29]:
import datetime as dt
march_date_crawled = autos.loc[(autos['date_crawled'] >= dt.date(2016,3,1)) & (autos['date_crawled'] <= dt.date(2016,4,1)),'date_crawled'].value_counts(normalize=True, dropna=False)

In [30]:
march_date_crawled.sort_index()

2016-03-05    0.029053
2016-03-06    0.016109
2016-03-07    0.041311
2016-03-08    0.038194
2016-03-09    0.037957
2016-03-10    0.036918
2016-03-11    0.037367
2016-03-12    0.042351
2016-03-13    0.017975
2016-03-14    0.041926
2016-03-15    0.039327
2016-03-16    0.033966
2016-03-17    0.036280
2016-03-18    0.014810
2016-03-19    0.039894
2016-03-20    0.043461
2016-03-21    0.042870
2016-03-22    0.037839
2016-03-23    0.036965
2016-03-24    0.033659
2016-03-25    0.036257
2016-03-26    0.036942
2016-03-27    0.035666
2016-03-28    0.039989
2016-03-29    0.039115
2016-03-30    0.038642
2016-03-31    0.036517
2016-04-01    0.038642
Name: date_crawled, dtype: float64

sample percentage distribution of march by date_crawled column

In [31]:
march_last_seen = autos.loc[(autos['last_seen'] > dt.date(2016,3,1)) & (autos['last_seen'] < dt.date(2016,4,1)),'last_seen'].value_counts(normalize=True, dropna=False)

In [32]:
march_last_seen.sort_index()

2016-03-05    0.002525
2016-03-06    0.010196
2016-03-07    0.012721
2016-03-08    0.017479
2016-03-09    0.022626
2016-03-10    0.025151
2016-03-11    0.029180
2016-03-12    0.056079
2016-03-13    0.020975
2016-03-14    0.029715
2016-03-15    0.037434
2016-03-16    0.038794
2016-03-17    0.066226
2016-03-18    0.017333
2016-03-19    0.037337
2016-03-20    0.048699
2016-03-21    0.048650
2016-03-22    0.050398
2016-03-23    0.043698
2016-03-24    0.046611
2016-03-25    0.045300
2016-03-26    0.039619
2016-03-27    0.036900
2016-03-28    0.049184
2016-03-29    0.052680
2016-03-30    0.058409
2016-03-31    0.056079
Name: last_seen, dtype: float64

Sample of distribution percentage of march last_seen logs

In [33]:
march_ad_created = autos.loc[(autos['ad_created'] > dt.date(2016,3,1)) & (autos['ad_created'] < dt.date(2016,4,1)),'ad_created'].value_counts(normalize=True,dropna=False)

In [34]:
march_ad_created.sort_index()

2016-03-02    0.000123
2016-03-03    0.001033
2016-03-04    0.001770
2016-03-05    0.027343
2016-03-06    0.018294
2016-03-07    0.041482
2016-03-08    0.039786
2016-03-09    0.039589
2016-03-10    0.038089
2016-03-11    0.039294
2016-03-12    0.043892
2016-03-13    0.020311
2016-03-14    0.042023
2016-03-15    0.040622
2016-03-16    0.035974
2016-03-17    0.037351
2016-03-18    0.016229
2016-03-19    0.040228
2016-03-20    0.045318
2016-03-21    0.044876
2016-03-22    0.039171
2016-03-23    0.038286
2016-03-24    0.034966
2016-03-25    0.037917
2016-03-26    0.038532
2016-03-27    0.037007
2016-03-28    0.041777
2016-03-29    0.040646
2016-03-30    0.040007
2016-03-31    0.038064
Name: ad_created, dtype: float64

sample of percentage distribution of march ad_created logs

## Summary of exploratory date_crawled,  ad_created, and last_seen logs
+ sorted autos by date_crawled, ad_created and last_seen from ascending order
+ changed last_seen and date_crawled columns to datetime date objects instead of time stamp for percentage distrubutions
+ provided some sample percentages distribution logs for each column

In [35]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The year that the car was first registered will likely indicate the age of the car. Looking at this column, we note some odd values. The minimum value is 1000, long before cars were invented and the maximum is 9999, many years into the future.

In [36]:
autos.registration_year.sort_values(ascending=False)

33950    9999
8012     9999
38076    9999
49910    9000
25003    8888
8360     6200
27618    5911
24519    5000
4164     5000
22799    5000
49153    5000
42079    4800
453      4500
4549     4100
27578    2800
5763     2019
49185    2019
33091    2018
6748     2018
41372    2018
22550    2018
3716     2018
47809    2018
33999    2018
49218    2018
20978    2018
20377    2018
13244    2018
45825    2018
11264    2018
         ... 
44406    1951
35921    1951
32091    1950
14020    1950
1171     1950
11047    1948
11585    1943
13963    1941
25792    1941
24855    1939
26103    1938
23804    1937
21421    1937
39725    1937
26607    1937
2221     1934
2573     1934
11246    1931
22101    1929
21416    1927
45157    1910
30781    1910
3679     1910
28693    1910
22659    1910
10556    1800
32585    1800
24511    1111
49283    1001
22316    1000
Name: registration_year, Length: 48565, dtype: int64

In [37]:
autos['registration_year'].between(1900,2018).sum()/autos.shape[0]

0.9995469988674972

 contrary to dataquest solution, we may be using an updated version of the original data set, since our registration_year column has 96% listings between 1900 - 2016

## Dealing with Incorrect Registration Year Data¶
Because a car can't be first registered before the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

One option is to remove the listings with these values. Let's determine what percentage of our data has invalid values in this column:

In [38]:
autos = autos[autos['registration_year'].between(1900,2020)].sort_values(by=['registration_year'],ascending=False)

In [39]:
autos.tail()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
22659,2016-03-14,Opel_Corsa_B,500,test,NaN,1910,NaN,0,corsa,150000,0,NaN,opel,NaN,2016-03-14,52393,2016-04-03
28693,2016-03-22,Renault_Twingo,599,control,kleinwagen,1910,manuell,0,NaN,5000,0,benzin,renault,NaN,2016-03-22,70376,2016-04-06
3679,2016-04-04,Suche_Auto,1,test,NaN,1910,NaN,0,NaN,5000,0,NaN,sonstige_autos,NaN,2016-04-04,40239,2016-04-04
45157,2016-03-11,Motorhaube,15,control,NaN,1910,NaN,0,NaN,5000,0,NaN,trabant,NaN,2016-03-11,90491,2016-03-25
30781,2016-03-25,Opel_Calibra_V6_DTM_Bausatz_1:24,30,test,NaN,1910,NaN,0,calibra,100000,0,NaN,opel,NaN,2016-03-25,47638,2016-03-26


simply took out the outrageously irrelevant entries which comprised of lesson that 1% of the data


In [40]:
autos.registration_year.value_counts(normalize=True)

2000    0.065012
2005    0.060480
1999    0.059677
2004    0.055680
2003    0.055598
2006    0.055001
2001    0.054300
2002    0.051210
1998    0.048676
2007    0.046905
2008    0.045628
2009    0.042950
1997    0.040190
2011    0.033433
2010    0.032733
2017    0.028674
1996    0.028283
2012    0.026985
1995    0.025276
2016    0.025131
2013    0.016541
2014    0.013657
1994    0.012957
2018    0.009682
1993    0.008755
2015    0.008075
1992    0.007622
1990    0.007148
1991    0.006983
1989    0.003584
          ...   
1976    0.000433
1969    0.000391
1975    0.000371
1965    0.000350
1964    0.000247
1963    0.000165
1959    0.000124
1961    0.000124
1910    0.000103
1937    0.000082
1962    0.000082
1958    0.000082
1956    0.000082
1950    0.000062
1955    0.000041
2019    0.000041
1934    0.000041
1957    0.000041
1951    0.000041
1941    0.000041
1954    0.000041
1953    0.000021
1948    0.000021
1927    0.000021
1943    0.000021
1929    0.000021
1939    0.000021
1938    0.0000

it seems a large percentage of car manufactering occured between 2000 - 2010

In [41]:
# autos.registration_year = pd.to_datetime(autos.registration_year,format="%Y")

In [42]:
# autos.registration_year = pd.DatetimeIndex(autos.registration_year).year

In [43]:
autos.brand.value_counts(normalize=True).head(20)

volkswagen        0.212813
bmw               0.108641
opel              0.108641
mercedes_benz     0.095787
audi              0.085858
ford              0.069667
renault           0.047894
peugeot           0.029457
fiat              0.025996
seat              0.018931
skoda             0.016068
nissan            0.015264
mazda             0.015223
smart             0.014296
citroen           0.014111
toyota            0.012586
hyundai           0.009950
sonstige_autos    0.009641
volvo             0.009043
mini              0.008611
Name: brand, dtype: float64

The Top 5 brands of cars are volkswagen,opel, bmw, mercedez benz, and audi. They comprise of roughly 50 % of the share in entries in the brand column.

In [44]:
dic = {}
for brand in autos.brand.value_counts(normalize=True):
    volkswagen = autos[autos['brand'] == 'volkswagen']
    opel = autos[autos['brand'] == 'opel']
    bmw = autos[autos['brand'] == 'bmw']
    mercedez_benz = autos[autos['brand'] == 'mercedes_benz']
    audi = autos[autos['brand'] == 'audi']
    volks_avg = volkswagen.price.sum() / volkswagen.shape[0]
    dic['volkswagen'] = volks_avg
    opel_avg = opel.price.sum()/opel.shape[0]
    dic['opel'] = opel_avg
    bmw_avg = bmw.price.sum()/bmw.shape[0]
    dic['bmw'] = bmw_avg
    mercedes_avg = mercedez_benz.price.sum()/mercedez_benz.shape[0]
    dic['mercedes_benz'] = mercedes_avg
    audi_avg = audi.price.sum()/audi.shape[0]
    dic['audi'] = audi_avg
    

In [45]:
print(dic)

{'mercedes_benz': 8526.623225806452, 'volkswagen': 5333.1962055948115, 'opel': 2941.4664391353813, 'bmw': 8261.382442169132, 'audi': 9212.9306621881}


+ audi: 9212
+ mercedes_benz: 8526
+ volkswagen: 5333
+ bmw: 8261
+ opel: 2941

In [46]:
mean_mileage = {}
for brand in autos.brand.value_counts(normalize=True):
    volkswagen = autos[autos['brand'] == 'volkswagen']
    opel = autos[autos['brand'] == 'opel']
    bmw = autos[autos['brand'] == 'bmw']
    mercedez_benz = autos[autos['brand'] == 'mercedes_benz']
    audi = autos[autos['brand'] == 'audi']
    mean_mileage['volkswagen'] = volkswagen.odometer_km.sum()/volkswagen.shape[0]
    mean_mileage['opel'] = opel.odometer_km.sum()/opel.shape[0]
    mean_mileage['bmw'] = bmw.odometer_km.sum()/opel.shape[0]
    mean_mileage['mercedes_benz'] = mercedez_benz.odometer_km.sum()/mercedez_benz.shape[0]
    mean_mileage['audi'] = audi.odometer_km.sum()/audi.shape[0]
    
    

In [47]:
print(mean_mileage)

{'mercedes_benz': 130848.3870967742, 'volkswagen': 128955.570612719, 'opel': 129452.02882062951, 'bmw': 132682.97307546454, 'audi': 129492.56238003839}


In [48]:
dic = pd.Series(dic)
mean_mileage = pd.Series(mean_mileage)

In [49]:
dic = pd.DataFrame(dic,columns=['mean_price'])
mean_mileage = pd.DataFrame(mean_mileage,columns=['mean_mileage'])

In [50]:
dic

,mean_price
audi,9212.930662
bmw,8261.382442
mercedes_benz,8526.623226
opel,2941.466439
volkswagen,5333.196206


In [51]:
mean_mileage

,mean_mileage
audi,129492.562380
bmw,132682.973075
mercedes_benz,130848.387097
opel,129452.028821
volkswagen,128955.570613


In [52]:
dic.join(mean_mileage)

,mean_price,mean_mileage
audi,9212.930662,129492.562380
bmw,8261.382442,132682.973075
mercedes_benz,8526.623226,130848.387097
opel,2941.466439,129452.028821
volkswagen,5333.196206,128955.570613


There is no particular relationship between the car brands pricing and its mileage in usage

## Identify categorical data that uses german words
+ translate them, map the values to their english counterparts.


In [53]:
autos.unrepaired_damage.value_counts(normalize=True)

nein    0.881171
ja      0.118829
Name: unrepaired_damage, dtype: float64

In [54]:
autos.unrepaired_damage = ['no' if i == 'nein' else 'yes' if i == 'ja' else i for i in autos.unrepaired_damage]

In [55]:
autos.gearbox.value_counts(normalize=True)

manuell      0.781084
automatik    0.218916
Name: gearbox, dtype: float64

In [56]:
autos.gearbox = ['manual' if i == 'manuell' else 'automatic' if i == 'automatik' else i for i in autos.gearbox]

In [57]:
autos.fuel_type.value_counts(normalize=True)

benzin     0.659413
diesel     0.322307
lpg        0.015001
cng        0.001639
hybrid     0.000831
elektro    0.000427
andere     0.000382
Name: fuel_type, dtype: float64

In [58]:
autos.fuel_type = autos.fuel_type.replace({'benzin':'gasoline','elektro':'electricity','andere':'other'})


In [59]:
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
49185,2016-03-28,Alfa_Romeo_156_Sportwagon_1_8_T_Spark,1200,control,NaN,2019,manual,140,156,150000,5,gasoline,alfa_romeo,no,2016-03-28,91058,2016-04-06
5763,2016-03-05,Suche_Auspuff_Anlage_honda_crx_del_sol,100,test,NaN,2019,manual,125,cr_reihe,20000,4,NaN,honda,no,2016-03-05,18106,2016-03-06
47258,2016-03-18,Jaguar_xj40,3500,control,NaN,2018,automatic,222,x_type,150000,4,NaN,jaguar,no,2016-03-18,89269,2016-04-05
38345,2016-03-28,Chevrolet_Aveo_LT___euro_4__1399_cc__TÜV_neu_....,2199,control,NaN,2018,NaN,90,NaN,150000,11,NaN,chevrolet,NaN,2016-03-28,33378,2016-03-31
29864,2016-03-05,VW_Lupo_BJ_1999,700,control,NaN,2018,manual,75,lupo,150000,0,gasoline,volkswagen,yes,2016-03-05,49124,2016-03-11


## Convert dates to tbe uniform numeriuc data
+ e.g. '2016-03-21' becomes 20160321
+ see if there are particular keywords in the name column that you can extract as new columns

In [60]:
def to_integer(dt_time):
    int_array = [10000*i.year + 100*i.month + i.day for i in dt_time]
        
    return int_array

In [61]:
autos.date_crawled = to_integer(autos.date_crawled)


In [62]:
autos.date_crawled.head()

49185    20160328
5763     20160305
47258    20160318
38345    20160328
29864    20160305
Name: date_crawled, dtype: int64

In [63]:
autos.last_seen = to_integer(autos.last_seen)

In [64]:
autos.last_seen.head()

49185    20160406
5763     20160306
47258    20160405
38345    20160331
29864    20160311
Name: last_seen, dtype: int64

In [65]:
autos.ad_created = to_integer(autos.ad_created)

In [66]:
autos.ad_created.head()

49185    20160328
5763     20160305
47258    20160318
38345    20160328
29864    20160305
Name: ad_created, dtype: int64

## lets see if there are keywords in name column that can be extracted as new columns

In [67]:
autos.name.value_counts(normalize=True)

Ford_Fiesta                                                          0.001566
Volkswagen_Golf_1.4                                                  0.001545
BMW_316i                                                             0.001545
BMW_318i                                                             0.001483
Volkswagen_Polo                                                      0.001483
BMW_320i                                                             0.001463
Renault_Twingo                                                       0.001421
Opel_Corsa                                                           0.001421
Volkswagen_Golf                                                      0.001174
Opel_Corsa_1.2_16V                                                   0.001154
BMW_116i                                                             0.001092
Peugeot_206                                                          0.001030
Ford_Focus                                                      

In [68]:
autos.sort_values(by='name')

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
46266,20160314,!!!!!!!!!!!!!!Renault_Clio_5_Tuerig_Servolenku...,100,control,kleinwagen,1995,manual,60,clio,150000,8,gasoline,renault,no,20160314,53757,20160317
35776,20160305,!!!!!!BMW_325_COUPE_TOP_ZUSTAND_ANSCHAUEN_LOHN...,4549,control,coupe,2000,manual,192,3er,150000,11,gasoline,bmw,no,20160305,13353,20160322
18849,20160329,!!!!!VW_PASSAT_3C_2.0TDI!!!!_SEHR_GEPFLEGT,8399,control,kombi,2006,automatic,140,passat,150000,3,diesel,volkswagen,NaN,20160329,99734,20160406
23991,20160316,!!!!!_Mittwoch_Angebot_!!!!_Nissan_Micra_1.2_A...,5250,test,kleinwagen,2011,manual,80,micra,100000,3,gasoline,nissan,no,20160316,48527,20160317
4017,20160314,!!!!!_Montag_Angebot_!!!!_Nissan_Micra_1.2_Ace...,5250,control,kleinwagen,2011,manual,80,micra,100000,3,gasoline,nissan,no,20160314,48527,20160316
18421,20160324,!!!!!_Ostern_Angebot_!!!!_Nissan_Micra_1.2_Ace...,5248,test,kleinwagen,2011,manual,80,micra,100000,3,gasoline,nissan,no,20160324,48527,20160326
3528,20160316,!!!!!_Skoda_Octavia_1.9_TDI_DPF_Ambiente_!!!!!,7600,test,limousine,2009,manual,105,octavia,70000,6,diesel,skoda,yes,20160316,26203,20160319
46858,20160307,!!!!ACHTUNG_FORD_ESCORT_TÜV_NEU_!!!!!!,1080,control,kleinwagen,1997,manual,75,escort,150000,6,gasoline,ford,no,20160307,39130,20160309
34092,20160329,!!!!Gepflegert_Fiat_Punto_1_Hand_!!!!!,1200,control,NaN,2005,manual,0,punto,125000,0,gasoline,fiat,NaN,20160329,90530,20160406
17193,20160309,!!!!VW_Polo_mit_neuem_TÜV_!!!,1800,test,kleinwagen,2000,manual,50,NaN,150000,7,gasoline,volkswagen,NaN,20160309,88416,20160309


sort values of name column via .sort_values

In [69]:
import re
autos.name.groupby(autos.name.str.extract("^[A-Za-z0-9_-]*$"),axis=1)

/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  from ipykernel import kernelapp as app


ValueError: pattern contains no capture groups

conventional groupby method says there is no pattern for capture group.
will try to hard code this.

In [70]:
autos.name = ['volkswagen' if 'Volks' in i else 'bmw' if 'BMW' in i else 'mercedes' if 'Mercedes' in i else 'Renault' if 'Renault' in i else 'ford' if 'Ford' in i else 'Andere' if 'Andere' in i else 'opel' if 'Opel' in i else 'audi' if 'Audi' in i else 'oldtimer' if 'Oldtimer' in i else i for i in autos.name]


In [71]:
autos.name

49185                Alfa_Romeo_156_Sportwagon_1_8_T_Spark
5763                Suche_Auspuff_Anlage_honda_crx_del_sol
47258                                          Jaguar_xj40
38345    Chevrolet_Aveo_LT___euro_4__1399_cc__TÜV_neu_....
29864                                      VW_Lupo_BJ_1999
21295                                             mercedes
32160                                              Renault
42396                                              Renault
46451                                  Fiat_Ducato_2.8.JTD
41656    Seat_Arosa_1_4_Scheckheft_gepflegt_kein_wartun...
24153                                       1.4tdi_tuf_neu
26191                                                  bmw
32600                                Golf_cabrio_1.6_90_ps
6675                           C220_esprit_tausch_moeglich
18727                                           volkswagen
20297                                    Vw_Passat_1.9_Tdi
19523                                                  b

In [72]:
autos.sort_values(by='name')

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
18849,20160329,!!!!!VW_PASSAT_3C_2.0TDI!!!!_SEHR_GEPFLEGT,8399,control,kombi,2006,automatic,140,passat,150000,3,diesel,volkswagen,NaN,20160329,99734,20160406
23991,20160316,!!!!!_Mittwoch_Angebot_!!!!_Nissan_Micra_1.2_A...,5250,test,kleinwagen,2011,manual,80,micra,100000,3,gasoline,nissan,no,20160316,48527,20160317
4017,20160314,!!!!!_Montag_Angebot_!!!!_Nissan_Micra_1.2_Ace...,5250,control,kleinwagen,2011,manual,80,micra,100000,3,gasoline,nissan,no,20160314,48527,20160316
18421,20160324,!!!!!_Ostern_Angebot_!!!!_Nissan_Micra_1.2_Ace...,5248,test,kleinwagen,2011,manual,80,micra,100000,3,gasoline,nissan,no,20160324,48527,20160326
3528,20160316,!!!!!_Skoda_Octavia_1.9_TDI_DPF_Ambiente_!!!!!,7600,test,limousine,2009,manual,105,octavia,70000,6,diesel,skoda,yes,20160316,26203,20160319
46858,20160307,!!!!ACHTUNG_FORD_ESCORT_TÜV_NEU_!!!!!!,1080,control,kleinwagen,1997,manual,75,escort,150000,6,gasoline,ford,no,20160307,39130,20160309
34092,20160329,!!!!Gepflegert_Fiat_Punto_1_Hand_!!!!!,1200,control,NaN,2005,manual,0,punto,125000,0,gasoline,fiat,NaN,20160329,90530,20160406
17193,20160309,!!!!VW_Polo_mit_neuem_TÜV_!!!,1800,test,kleinwagen,2000,manual,50,NaN,150000,7,gasoline,volkswagen,NaN,20160309,88416,20160309
7942,20160405,!!!Familien_kombi_lauft_sehr_gut_zu_verlassig_!!!,999,test,kombi,1999,manual,0,c_klasse,150000,7,gasoline,mercedes_benz,NaN,20160405,12169,20160407
5540,20160331,!!!Rentner_sucht_kombi_oder_golf_4!!!,1000,test,NaN,2018,automatic,122,c_klasse,5000,5,NaN,mercedes_benz,NaN,20160331,12169,20160406


nope nope. bad idea